In [82]:
import torch
import torch.nn as nn
import numpy as np

In [119]:
class SoundNet(nn.Module):
    def __init__(self):
        super(SoundNet, self).__init__()

        self.conv1      = nn.Conv1d(1, 16, 64, stride=2, padding=32)
        print("Conv1", self.conv1.weight.shape, self.conv1.bias.shape)
        self.batchnorm1 = nn.BatchNorm1d(16)
        print("Bn1", self.batchnorm1.weight.shape, self.batchnorm1.bias.shape)
        self.relu1      = nn.ReLU(True)
        self.maxpool1   = nn.MaxPool1d(8, stride=8)

        self.conv2      = nn.Conv1d(16, 32, 32, stride=2, padding=16)
        print("Conv2", self.conv2.weight.shape, self.conv2.bias.shape)
        self.batchnorm2 = nn.BatchNorm1d(32)
        print("Bn2", self.batchnorm2.weight.shape, self.batchnorm2.bias.shape)
        self.relu2      = nn.ReLU(True)
        self.maxpool2   = nn.MaxPool1d(8, stride=8)

        self.conv3      = nn.Conv1d(32, 64, 16, stride=2, padding=8)
        print("Conv3", self.conv3.weight.shape, self.conv3.bias.shape)
        self.batchnorm3 = nn.BatchNorm1d(64)
        print("Bn3", self.batchnorm3.weight.shape, self.batchnorm3.bias.shape)
        self.relu3      = nn.ReLU(True)

        self.conv4      = nn.Conv1d(64, 128, 8, stride=2, padding=4)
        print("Conv4", self.conv4.weight.shape, self.conv4.bias.shape)
        self.batchnorm4 = nn.BatchNorm1d(128)
        print("Bn4", self.batchnorm4.weight.shape, self.batchnorm4.bias.shape)
        self.relu4      = nn.ReLU(True)

        self.conv5      = nn.Conv1d(128, 256, 4, stride=2, padding=2)
        print("Conv5", self.conv5.weight.shape, self.conv5.bias.shape)
        self.batchnorm5 = nn.BatchNorm1d(256)
        print("Bn5", self.batchnorm5.weight.shape, self.batchnorm5.bias.shape)
        self.relu5      = nn.ReLU(True)
        self.maxpool5   = nn.MaxPool1d(4, stride=4)

        self.conv6      = nn.Conv1d(256, 512, 4, stride=2, padding=2)
        print("Conv6", self.conv6.weight.shape, self.conv6.bias.shape)
        self.batchnorm6 = nn.BatchNorm1d(512)
        print("Bn6", self.batchnorm6.weight.shape, self.batchnorm6.bias.shape)
        self.relu6      = nn.ReLU(True)

        self.conv7      = nn.Conv1d(512, 1024, 4, stride=2, padding=2)
        print("Conv7", self.conv7.weight.shape, self.conv7.bias.shape)
        self.batchnorm7 = nn.BatchNorm1d(1024)
        print("Bn7", self.batchnorm7.weight.shape, self.batchnorm7.bias.shape)
        self.relu7      = nn.ReLU(True)

        # replace the last layer
#         self.conv8_objs = nn.Conv1d(1024, 1000, 8, stride=2)
#         print("Conv81", self.conv8_objs.weight.shape, self.conv8_objs.bias.shape)
#         self.conv8_scns = nn.Conv1d(1024, 401,  8, stride=2)
#         print("Conv82", self.conv8_scns.weight.shape, self.conv8_scns.bias.shape)

        self.conv8      = nn.Conv1d(1024, 256, 8)
        print("Conv8", self.conv8.weight.shape, self.conv8.bias.shape)
#         self.batchnorm8 = nn.BatchNorm1d(256)
#         print("Bn8", self.batchnorm8.weight.shape, self.batchnorm8.bias.shape)
        self.relu8      = nn.ReLU(True) 
    
    
    def forward(self, waveform):
        """
            Args:
                waveform (Variable): Raw 10s waveform.
        """
        if torch.cuda.is_available():
            waveform.cuda()

        print("Size of input: ", waveform.size())
        out = self.conv1(waveform)
        out = self.batchnorm1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)
        print("Size after layer 1: ", out.size())
        
        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        print("Size after layer 2: ", out.size())
        
        out = self.conv3(out)
        out = self.batchnorm3(out)
        out = self.relu3(out)
        print("Size after layer 3: ", out.size())
        
        out = self.conv4(out)
        out = self.batchnorm4(out)
        out = self.relu4(out)
        print("Size after layer 4: ", out.size())
        
        out = self.conv5(out)
        out = self.batchnorm5(out)
        out = self.relu5(out)
        out = self.maxpool5(out)
        print("Size after layer 5: ", out.size())
        
        out = self.conv6(out)
        out = self.batchnorm6(out)
        out = self.relu6(out)
        print("Size after layer 6: ", out.size())

        out = self.conv7(out)
        out = self.batchnorm7(out)
        out = self.relu7(out)
        print("Size after layer 7: ", out.size())

        # replace
#         p_objs = self.conv8_objs(out)
#         p_scns = self.conv8_scns(out)
#         print("Size after OBJS: ", p_objs.size())
#         print("Size after SCNS: ", p_scns.size())

#         return (nn.Softmax(dim=1)(p_objs), nn.Softmax(dim=1)(p_scns))
            
        out = self.conv8(out)
        #out = self.batchnorm8(out)
        out = self.relu8(out)
        print("Size after layer 8: ", out.size())
        
        return out
    

    def load_weights(self):
        bn1_bs = np.load('bn1_bs.npy')
        self.batchnorm1.bias = torch.nn.Parameter(torch.from_numpy(bn1_bs))
        bn1_ws = np.load('bn1_ws.npy')
        self.batchnorm1.weight = torch.nn.Parameter(torch.from_numpy(bn1_ws))
        bn2_bs = np.load('bn2_bs.npy')
        self.batchnorm2.bias = torch.nn.Parameter(torch.from_numpy(bn2_bs))
        bn2_ws = np.load('bn2_ws.npy')
        self.batchnorm2.weight = torch.nn.Parameter(torch.from_numpy(bn2_ws))
        bn3_bs = np.load('bn3_bs.npy')
        self.batchnorm3.bias = torch.nn.Parameter(torch.from_numpy(bn3_bs))
        bn3_ws = np.load('bn3_ws.npy')
        self.batchnorm3.weight = torch.nn.Parameter(torch.from_numpy(bn3_ws))
        bn4_bs = np.load('bn4_bs.npy')
        self.batchnorm4.bias = torch.nn.Parameter(torch.from_numpy(bn4_bs))
        bn4_ws = np.load('bn4_ws.npy')
        self.batchnorm4.weight = torch.nn.Parameter(torch.from_numpy(bn4_ws))
        bn5_bs = np.load('bn5_bs.npy')
        self.batchnorm5.bias = torch.nn.Parameter(torch.from_numpy(bn5_bs))
        bn5_ws = np.load('bn5_ws.npy')
        self.batchnorm5.weight = torch.nn.Parameter(torch.from_numpy(bn5_ws))
        bn6_bs = np.load('bn6_bs.npy')
        self.batchnorm6.bias = torch.nn.Parameter(torch.from_numpy(bn6_bs))
        bn6_ws = np.load('bn6_ws.npy')
        self.batchnorm6.weight = torch.nn.Parameter(torch.from_numpy(bn6_ws))
        bn7_bs = np.load('bn7_bs.npy')
        self.batchnorm7.bias = torch.nn.Parameter(torch.from_numpy(bn7_bs))
        bn7_ws = np.load('bn7_ws.npy')
        self.batchnorm7.weight = torch.nn.Parameter(torch.from_numpy(bn7_ws))

        conv1_bs = np.load('conv1_bs.npy')
        self.conv1.bias = torch.nn.Parameter(torch.from_numpy(conv1_bs))
        conv1_ws = np.load('conv1_ws.npy')
        conv1_ws = np.reshape(conv1_ws, self.conv1.weight.shape)
        self.conv1.weight = torch.nn.Parameter(torch.from_numpy(conv1_ws))

        conv2_bs = np.load('conv2_bs.npy')
        self.conv2.bias = torch.nn.Parameter(torch.from_numpy(conv2_bs))
        conv2_ws = np.load('conv2_ws.npy')
        conv2_ws = np.reshape(conv2_ws, self.conv2.weight.shape)
        self.conv2.weight = torch.nn.Parameter(torch.from_numpy(conv2_ws))

        conv3_bs = np.load('conv3_bs.npy')
        self.conv3.bias = torch.nn.Parameter(torch.from_numpy(conv3_bs))
        conv3_ws = np.load('conv3_ws.npy')
        conv3_ws = np.reshape(conv3_ws, self.conv3.weight.shape)
        self.conv3.weight = torch.nn.Parameter(torch.from_numpy(conv3_ws))

        conv4_bs = np.load('conv4_bs.npy')
        self.conv4.bias = torch.nn.Parameter(torch.from_numpy(conv4_bs))
        conv4_ws = np.load('conv4_ws.npy')
        conv4_ws = np.reshape(conv4_ws, self.conv4.weight.shape)
        self.conv4.weight = torch.nn.Parameter(torch.from_numpy(conv4_ws))

        conv5_bs = np.load('conv5_bs.npy')
        self.conv5.bias = torch.nn.Parameter(torch.from_numpy(conv5_bs))
        conv5_ws = np.load('conv5_ws.npy')
        conv5_ws = np.reshape(conv5_ws, self.conv5.weight.shape)
        self.conv5.weight = torch.nn.Parameter(torch.from_numpy(conv5_ws))

        conv6_bs = np.load('conv6_bs.npy')
        self.conv6.bias = torch.nn.Parameter(torch.from_numpy(conv6_bs))
        conv6_ws = np.load('conv6_ws.npy')
        conv6_ws = np.reshape(conv6_ws, self.conv6.weight.shape)
        self.conv6.weight = torch.nn.Parameter(torch.from_numpy(conv6_ws))

        conv7_bs = np.load('conv7_bs.npy')
        self.conv7.bias = torch.nn.Parameter(torch.from_numpy(conv7_bs))
        conv7_ws = np.load('conv7_ws.npy')
        conv7_ws = np.reshape(conv7_ws, self.conv7.weight.shape)
        self.conv7.weight = torch.nn.Parameter(torch.from_numpy(conv7_ws))

        # replace
#         conv81_bs = np.load('conv81_bs.npy')
#         self.conv8_objs.bias = torch.nn.Parameter(torch.from_numpy(conv81_bs))
#         conv81_ws = np.load('conv81_ws.npy')
#         conv81_ws = np.reshape(conv81_ws, self.conv8_objs.weight.shape)
#         self.conv8_objs.weight = torch.nn.Parameter(torch.from_numpy(conv81_ws))

#         conv82_bs = np.load('conv82_bs.npy')
#         self.conv8_scns.bias = torch.nn.Parameter(torch.from_numpy(conv82_bs))
#         conv82_ws = np.load('conv82_ws.npy')
#         conv82_ws = np.reshape(conv82_ws, self.conv8_scns.weight.shape)
#         self.conv8_scns.weight = torch.nn.Parameter(torch.from_numpy(conv82_ws))

In [120]:
soundnet = SoundNet()
soundnet.load_weights()

Conv1 torch.Size([16, 1, 64]) torch.Size([16])
Bn1 torch.Size([16]) torch.Size([16])
Conv2 torch.Size([32, 16, 32]) torch.Size([32])
Bn2 torch.Size([32]) torch.Size([32])
Conv3 torch.Size([64, 32, 16]) torch.Size([64])
Bn3 torch.Size([64]) torch.Size([64])
Conv4 torch.Size([128, 64, 8]) torch.Size([128])
Bn4 torch.Size([128]) torch.Size([128])
Conv5 torch.Size([256, 128, 4]) torch.Size([256])
Bn5 torch.Size([256]) torch.Size([256])
Conv6 torch.Size([512, 256, 4]) torch.Size([512])
Bn6 torch.Size([512]) torch.Size([512])
Conv7 torch.Size([1024, 512, 4]) torch.Size([1024])
Bn7 torch.Size([1024]) torch.Size([1024])
Conv8 torch.Size([256, 1024, 8]) torch.Size([256])


In [130]:
LEN_WAVEFORM = 44100*5
import torchaudio
data, sample_rate = torchaudio.load('/nfs2/datasets/TED/audio/WadeDavis_2008-480p.mp3')
data.size(), sample_rate

#data1 = data[:LEN_WAVEFORM]
data1 = data[:LEN_WAVEFORM]

In [131]:
channel_mean_data1 = torch.mean(data1,dim=1)
print(channel_mean_data1.size())
# channel_mean_data2 = torch.mean(data2,dim=1)
# print(channel_mean_data2.size())

torch.Size([220500])


In [132]:
channel_mean_data1 = channel_mean_data1.view(1,1,channel_mean_data1.size(0))
print(channel_mean_data1.size())
# channel_mean_data2 = channel_mean_data2.view(1,1,channel_mean_data2.size(0))
# print(channel_mean_data2.size())

torch.Size([1, 1, 220500])


In [133]:
output1 = soundnet(channel_mean_data1)
# output2 = soundnet(channel_mean_data2)

Size of input:  torch.Size([1, 1, 220500])
Size after layer 1:  torch.Size([1, 16, 13781])
Size after layer 2:  torch.Size([1, 32, 861])
Size after layer 3:  torch.Size([1, 64, 431])
Size after layer 4:  torch.Size([1, 128, 216])
Size after layer 5:  torch.Size([1, 256, 27])
Size after layer 6:  torch.Size([1, 512, 14])
Size after layer 7:  torch.Size([1, 1024, 8])
Size after layer 8:  torch.Size([1, 256, 1])


In [125]:
print(output1.size())
print(output1.size())
# print(output2[0].size())
# print(output2[1].size())

torch.Size([1, 256, 1])
torch.Size([1, 256, 1])


In [69]:
class PoseNet(nn.Module):
    def __init__(self):
        super(PoseNet, self).__init__()

        self.conv1      = nn.Conv1d(36, 64, 10, stride=1, padding=5)
        print("Conv1", self.conv1.weight.shape, self.conv1.bias.shape)    
        self.relu1      = nn.ReLU(True)
        

        self.conv2      = nn.Conv1d(64, 128, 10, stride=1, padding=5)
        print("Conv2", self.conv2.weight.shape, self.conv2.bias.shape)
        self.relu2      = nn.ReLU(True)
        

        self.conv3      = nn.Conv1d(128, 256, 10, stride=1, padding=5)
        print("Conv3", self.conv3.weight.shape, self.conv3.bias.shape)
        self.relu3      = nn.ReLU(True)

        
        self.conv4      = nn.Conv1d(256, 256, 10, stride=1, padding=5)
        print("Conv4", self.conv4.weight.shape, self.conv4.bias.shape)
        self.relu4      = nn.ReLU(True)


    def forward(self, pose):
        """
            Args:
                pose (Variable): Raw 10s pose information.
        """
        if torch.cuda.is_available():
            pose.cuda()

        print("Size of input: ", pose.size())
        out = self.conv1(pose)
        #out = self.batchnorm1(out)
        out = self.relu1(out)
        #out = self.maxpool1(out)
        print("Size after layer 1: ", out.size())
        
        out = self.conv2(out)
        #out = self.batchnorm2(out)
        out = self.relu2(out)
        #out = self.maxpool2(out)
        print("Size after layer 2: ", out.size())
        
        out = self.conv3(out)
        #out = self.batchnorm3(out)
        out = self.relu3(out)
        print("Size after layer 3: ", out.size())
        
        out = self.conv4(out)
        #out = self.batchnorm4(out)
        out = self.relu4(out)
        print("Size after layer 4: ", out.size())
        

In [79]:
posedata = torch.randn(36*300)
#posedata.size()
posedata = posedata.view(1,36,-1)
print(posedata.size())

torch.Size([1, 36, 300])


In [80]:
posenet = PoseNet()

Conv1 torch.Size([64, 36, 10]) torch.Size([64])
Conv2 torch.Size([128, 64, 10]) torch.Size([128])
Conv3 torch.Size([256, 128, 10]) torch.Size([256])
Conv4 torch.Size([256, 256, 10]) torch.Size([256])


In [81]:
output_pose = posenet(posedata)

Size of input:  torch.Size([1, 36, 300])
Size after layer 1:  torch.Size([1, 64, 301])
Size after layer 2:  torch.Size([1, 128, 302])
Size after layer 3:  torch.Size([1, 256, 303])
Size after layer 4:  torch.Size([1, 256, 304])


In [161]:
import subprocess

pose_only_frame_root = '/nfs2/datasets/TED/frames/AaronKoblin_2011-480p'
audiopath = '/nfs1/shared/for_aniruddha/pose/dataset/audios/AaronKoblin_2011-480p_0002.mp3'
store_path = '/nfs1/shared/for_aniruddha/regenerate_final_0003.mp4'
clip_video_path = '/nfs1/shared/for_aniruddha/regenerate_video_0003.mp4'
sequence_start_frame = "15481"


bash_command_video = "ffmpeg -framerate 30 -start_number " + sequence_start_frame + " -i " + pose_only_frame_root + "/%6d.jpg -vframes 150 -pix_fmt yuv420p -codec:v libx264 -y " + clip_video_path
print(bash_command_video)

subprocess.call(bash_command_video.split(' '))

bash_command1 = "ffmpeg -loglevel error -i " + clip_video_path + " -i " + audiopath + " -c:v copy -r 30 -c:a copy -strict experimental -y " + store_path
print(bash_command1)

subprocess.call(bash_command1.split(' '))

ffmpeg -framerate 30 -start_number 15481 -i /nfs2/datasets/TED/frames/AaronKoblin_2011-480p/%6d.jpg -vframes 150 -pix_fmt yuv420p -codec:v libx264 -y /nfs1/shared/for_aniruddha/regenerate_video_0003.mp4
ffmpeg -loglevel error -i /nfs1/shared/for_aniruddha/regenerate_video_0003.mp4 -i /nfs1/shared/for_aniruddha/pose/dataset/audios/AaronKoblin_2011-480p_0002.mp3 -c:v copy -r 30 -c:a copy -strict experimental -y /nfs1/shared/for_aniruddha/regenerate_final_0003.mp4


0

In [162]:
posenet.named_parameters()

<generator object Module.named_parameters at 0x7efcf950c410>

In [164]:
for tag, value in posenet.named_parameters():
    print (tag)
    print (value)

conv1.weight
Parameter containing:
tensor(1.00000e-02 *
       [[[-0.9049,  4.3081, -3.8603,  ..., -0.8333,  3.5384, -0.2840],
         [ 0.4630, -4.3452,  1.6706,  ...,  1.8270, -0.1649,  0.5262],
         [-1.5739, -3.2794,  2.9341,  ..., -2.1455, -2.2507, -4.0957],
         ...,
         [ 1.2406,  3.0575,  3.0486,  ...,  5.2100, -2.9382, -1.0612],
         [ 2.9407,  3.3743, -2.8915,  ..., -0.1833, -0.6028, -5.0985],
         [-2.9811,  1.8573,  2.4196,  ...,  2.2074, -2.5085,  2.7064]],

        [[-4.0294,  3.7837, -3.4412,  ...,  3.7236, -5.1823,  1.4328],
         [ 1.1438, -3.9206,  3.2194,  ..., -3.6679, -4.3261,  2.5304],
         [ 0.2853, -2.2933,  1.4477,  ...,  1.3271,  4.5030, -4.1439],
         ...,
         [ 1.0300,  0.2102, -3.2962,  ..., -4.9533, -4.3075,  4.7745],
         [-2.8435, -3.4318, -0.6136,  ..., -2.9708,  0.7202, -4.2107],
         [ 1.8962,  2.6117, -3.0563,  ..., -1.6439,  4.5801,  0.3899]],

        [[ 2.3897, -3.9763,  3.8594,  ...,  1.7878,  1.5926,